1. why p-1?
2. 是否需要存储公钥私钥然后再进行环签名
3. 改成DSA lib中的公私钥加密和随机数生成

In [871]:
import random
from hashlib import sha256

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

def schnorr_sign(message, g, p, s):
    # 选择随机数k
    k = random.randint(1, p-2)
    # 计算r = g^k mod p
    r = pow(g, k, p)
    # 将消息和r的值哈希，然后与私钥s结合产生签名
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    x = (k - s * hash_value) % (p-1)
    return r, x

def schnorr_verify(message, g, p, v, r, x):
    # 重新计算r的哈希值
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    # 使用公钥v和签名x、r计算验证值
    rv = (pow(g, x, p) * pow(v, hash_value, p)) % p
    return rv == r

# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

# 生成密钥
s, v = generate_keys(g, p)

# 签名消息
message = "Hello, Schnorr!"
r, x = schnorr_sign(message, g, p, s)
print(f"Signature: (r={r}, x={x})")

# 验证签名
valid = schnorr_verify(message, g, p, v, r, x)
print(f"Signature valid: {valid}")


Signature: (r=888814689, x=409963107)
Signature valid: True


In [872]:
import random 
from hashlib import sha256


p = 1000000007  # prime
g = 5  # primitive root of p

def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

# why p-1?
s2 = (r - x2 * e2) 
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

msg: Hello, Ring Signature
pubkeys: [820857965, 772694907, 327614128]
Y: 820857965772694907327614128
e3: 76589853696861692232659613050758639105027622924615322901656632481001887688153
random_s3: 460268367
e1: 66686734475520259752042849268097524157125678122490386557910217999171492950647
random_s1: 620932992
e2: 49943456389489943175509507242748409439543034128094499718755809048567381791710
s2: -40831037487604452156015497821020341549541537864189880791880636629039835182876728253973
signature process total result:


{'e1': 66686734475520259752042849268097524157125678122490386557910217999171492950647,
 's1': 620932992,
 's2': -40831037487604452156015497821020341549541537864189880791880636629039835182876728253973,
 's3': 460268367}

In [873]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
 
print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)
print(e2_V==e2)
print(e3_V==e3)

66686734475520259752042849268097524157125678122490386557910217999171492950647
49943456389489943175509507242748409439543034128094499718755809048567381791710
76589853696861692232659613050758639105027622924615322901656632481001887688153
True
True
True


In [874]:
e3_calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_calculation

751481223

In [875]:
e3_original_calculation = pow(g, r, p) % p
e3_original_calculation

751481223

In [876]:
(pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p

751481223

In [877]:
pow(g, r, p)

751481223